In [ ]:
# Install packages 
import os
import pandas as pd
import time
from transformers import pipeline
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import undetected_chromedriver as uc

In [ ]:
# URL of Forex Factory News
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled") 

In [ ]:
ua = UserAgent()
chrome_options.add_argument(f"user-agent={ua.random}")

In [ ]:
driver = uc.Chrome()
articles=[]
for i in range(0, 59):
    url = f"https://www.fxstreet.com/news?q=&hPP=17&idx=FxsIndexPro&p={i}"
    driver.get(url)
    time.sleep(3)
    response = driver.page_source
    soup = BeautifulSoup(response, "html.parser")
    for article in soup.find_all("div",class_="fxs_floatingMedia_textBody"):
        articles.append(article.find('a').get_text(strip=True))

In [ ]:
# Load FinBERT model
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")
# Function to classify sentiment using FinBERT
def classify_sentiment(text):
    result = finbert(text)[0] 
    return result["label"]  

In [ ]:
df = pd.read_csv("data/forex_news.csv")
df = pd.concat([df, pd.DataFrame({'news': articles, 'labels': list(map(classify_sentiment, articles))})], axis=0, ignore_index=True)
df = df.drop_duplicates().reset_index(drop=True)  

In [ ]:
df['news'] = df['news'].fillna('')
df=df[['news', 'labels']]
df.to_csv("data/forex_news.csv")